# Fairness auditing for subgroups using Fairness Aware Counterfactuals for Subgroups (FACTS).

[FACTS](https://arxiv.org/abs/2306.14978) is an efficient, model-agnostic, highly parameterizable, and explainable framework for evaluating subgroup fairness through counterfactual explanations.

In this notebook, we will see how to use this algorithm for discovering subgroups where the bias of a model (logistic regression for simplicity) between Males and Females is high.

We will use the Adult dataset from UCI ([reference](https://archive.ics.uci.edu/ml/datasets/adult)).

# Preliminaries

## Import dependencies

As usual in python, the first step is to import all necessary packages.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.datasets.openml_datasets import fetch_adult
from aif360.sklearn.detectors.facts.clean import clean_dataset
from aif360.sklearn.detectors.facts import FACTS

from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


Below, you can change the `random_seed` variable to `None` if you would like for the pseudo-random parts to actually change between runs. We have set it to a specific value for reproducibility.

In [2]:
random_seed = 131313 # for reproducibility

## Load Dataset

In [3]:
# load the adult dataset and perform some simple preprocessing steps
# See output for a glimpse of the final dataset's characteristics
X, y, sample_weight = fetch_adult()
data = clean_dataset(X.assign(income=y), "adult")
display(data.head())

# split into train-test data
y = data['income']
X = data.drop('income', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_seed, stratify=y)

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,"(16.999, 26.0]",Private,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,FullTime,United-States,0
1,"(34.0, 41.0]",Private,9.0,Married-civ-spouse,Farming-fishing,Married,White,Male,0.0,0.0,OverTime,United-States,0
2,"(26.0, 34.0]",Local-gov,12.0,Married-civ-spouse,Protective-serv,Married,White,Male,0.0,0.0,FullTime,United-States,1
3,"(41.0, 50.0]",Private,10.0,Married-civ-spouse,Machine-op-inspct,Married,Black,Male,7688.0,0.0,FullTime,United-States,1
4,"(26.0, 34.0]",Private,6.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,MidTime,United-States,0


## Example Model to be used for Auditing

We use the train set to train a simple logistic regression model. This will serve as the demonstrative model, which we will then treat as a black box and apply our algorithm.

Of course, any model can be used in its place. Our purpose here is not to produce a good model, but to audit the fairness of an existing one.

In [4]:
#### here, we incrementally build the example model. It consists of one preprocessing step,
#### which is to turn categorical features into the respective one-hot encodings, and
#### a simple scikit-learn logistic regressor.
categorical_features = X.select_dtypes(include=["object", "category"]).columns.to_list()
categorical_features_onehot_transformer = ColumnTransformer(
    transformers=[
        ("one-hot-encoder", OneHotEncoder(), categorical_features)
    ],
    remainder="passthrough"
)
model = Pipeline([
    ("one-hot-encoder", categorical_features_onehot_transformer),
    ("clf", LogisticRegression(max_iter=1500))
])

#### train the model
model = model.fit(X_train, y_train)

In [5]:
# showcase model's accuracy
y_pred = model.predict(X_test)
print(f"Accuracy = {(y_test.values == y_pred).sum() / y_test.shape[0]:.2%}")

Accuracy = 85.21%


# A Practical Example of FACTS

The real essence of our work starts here. Specifically, we showcase the generation of candidate subgroups and counterfactuals and the detection of those subgroups that exhibit the greatest unfairness, with respect to one of several metrics.

## Load and Fit FACTS

In [6]:
# load FACTS framework with:
# - the model to be audited
# - protected attribute "sex" and
# - assigning equal, unit weights to all features for cost computation.
detector = FACTS(
    estimator=model,
    prot_attr="sex",
    feature_weights={f: 1 for f in X.columns}
)

In [7]:
# generates candidate subpopulation groups for bias and candidate actions
detector = detector.fit(X_test)

Computing candidate subgroups.


100%|██████████████████████████████████████████████████████████████████████████| 1046/1046 [00:00<00:00, 520246.89it/s]

Number of subgroups: 568
Computing candidate recourses for all subgroups.



100%|█████████████████████████████████████████████████████████████████████████████| 568/568 [00:00<00:00, 42048.16it/s]

Computing percentages of individuals flipped by each action independently.



100%|████████████████████████████████████████████████████████████████████████████████| 608/608 [00:13<00:00, 44.59it/s]

Computing percentages of individuals flipped by any action with cost up to c, for every c



100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [00:12<00:00, 34.29it/s]


## Detect Groups with Unfairness in Protected Subgroups (using "Equal Choice for Recourse" metric)

Here we demonstrate the `bias_scan` method of our detector, which ranks subpopulation groups from most to least unfair, with respect to the chosen metric and, of course, the protected attribute.

For the purposes of this demo, we use the "Equal Choice for Recourse" metric. This metric claims that the classifier acts fairly for the group in question if the protected subgroups can choose among the same number of sufficiently effective actions to achieve recourse. By sufficiently effective we mean those actions (out of all candidates) which work for at least $100\phi \%$ (for $\phi \in [0,1]$) of the subgroup.

**Suggestion**: this metric may find utility in scenarios where the aim is to guarantee that protected subgroups have a similar range of options available to them when it comes to making adjustments in order to attain a favorable outcome. For example, when evaluating job candidates, the employer may wish ensure that applicants from different backgrounds (that currently fail to meet expectations) have an equal array of career / retraining options that land them the job; perhaps to ensure diversity in all sectors of the company, which employ individuals with a plethora of roles.

In [8]:
# Detects the top `top_count` most biased groups based on the given metric
# available metrics are:
# - equal-effectiveness
# - equal-choice-for-recourse
# - equal-effectiveness-within-budget
# - equal-cost-of-effectiveness
# - equal-mean-recourse
# - fair-tradeoff
# a short description for each metric is given below
detector.bias_scan(
    metric="equal-choice-for-recourse",
    phi=0.1,
    top_count=10
)

In [9]:
# prints the result into a nicely formatted report
detector.print_recourse_report(
    show_action_costs=True,
    show_subgroup_costs=True
)

If age = (26.0, 34.0], hours-per-week = FullTime:
	Protected Subgroup 'Female', 10.59% covered
		Make age = (41.0, 50.0], hours-per-week = OverTime with effectiveness 7.96% and counterfactual cost = 2.0.
		Make age = (41.0, 50.0] with effectiveness 4.22% and counterfactual cost = 1.0.
		Make age = (34.0, 41.0], hours-per-week = OverTime with effectiveness 5.15% and counterfactual cost = 2.0.
		Aggregate cost of the above recourses = 0.00
	Protected Subgroup 'Male', 13.81% covered
		Make age = (41.0, 50.0], hours-per-week = OverTime with effectiveness 19.72% and counterfactual cost = 2.0.
		Make age = (41.0, 50.0] with effectiveness 10.55% and counterfactual cost = 1.0.
		Make age = (34.0, 41.0], hours-per-week = OverTime with effectiveness 13.33% and counterfactual cost = 2.0.
		Aggregate cost of the above recourses = -3.00
	Bias against Female. Unfairness score = 3.
If hours-per-week = FullTime, race = White:
	Protected Subgroup 'Female', 36.14% covered
		Make hours-per-week = OverTim

# Short Description of all Definitions / Metrics of Subgroup Recourse Fairness

Here we give a brief description of each of the metrics available in our framework apart from "Equal Choice for Recourse".

## Equal Effectiveness

The classifier is considered to act fairly for a population group if the same proportion of individuals in the protected subgroups can achieve recourse.

**Suggestion**: ignores costs altogether and compares only the percentage of males VS females that can cross the model's decision boundary by the same actions. We would use it in applications where the goal is equality, in the sense that if members of protected subgroups perform the same change, they will see the same result (at least statistically). For example, in a hiring scenario where males and females are expected to have similar changes in their attributes correspond to similar results.

## Equal Effectiveness within Budget

The classifier is considered to act fairly for a population group if the same proportion of individuals in the protected subgroups can achieve recourse with a cost at most $c$, where $c$ is some user-provided cost budget.

**Suggestion**: similar to the above, but with a bound on how big we allow the cost of a recourse to be. Could be used as an alternative when changes with undesirably great cost are found. Continuing the previous example, if "Equal Effectiveness" finds a change of salary to 1000000 which makes everyone cross the decision boundary, we will not see bias, although with a smaller bound on the cost we might.

## Equal Cost of Effectiveness

The classifier is considered to act fairly for a population group if the minimum cost required to be sufficiently effective in the protected subgroups is equal. Again, as in "Equal Choice for Recourse", by "sufficiently effective" we refer to those actions that successfully flip the model's decision for at least $100\phi \%$ (for $\phi \in [0,1]$) of the subgroup.

**Suggestion**: could be useful when an external factor imposes a specific threshold, e.g. in credit risk assessment, a guideline which states that the effort required to be 80% certain that you will have your loan accepted should be the same for males and females.

## Equal (Conditional) Mean Recourse

This definition extends the notion of *burden* from literature ([reference](https://dl.acm.org/doi/10.1145/3375627.3375812)) to the case where not all individuals may achieve recourse. Omitting some details, given any set of individuals, the **conditional mean recourse cost** is the mean recourse cost among the subset of individuals that can actually achieve recourse, i.e. by at least one of the available actions.

Given the above, this definition considers the classifier to act fairly for a population group if the (conditional) mean recourse cost for the protected subgroups is the same.

**Suggestion**: this metric compares the mean costs required to achieve recourse for the protected subgroups. Thus, it could be useful in a scenario like loan approval, where the individuals potentially represent the entire populations. In such cases, one could consider that if e.g. males and females are met with the same difficulty, *in expectation*, to have their loan accepted, then the system can be considered fair.

## Fair Effectiveness-Cost Trade-Off

This is the strictest definition, which considers the classifier to act fairly for a population group only if the protected subgroups have the same effectiveness-cost distribution (checked in the implementation via a statistical test).

Equivalently, Equal Effectiveness within Budget must hold for *every* value of the cost budget $c$.

**Suggestion**: this definition essentially tries to consider all of the above, of course with some potential tradeoffs.